In [1]:
# A notebook with all the class and functions needed 

# general library osed for the fllowing functions and classes: 
import numpy as np
import pandas as pd 
import sklearn.preprocessing
import sklearn as sk 
import scipy as sy
import random

import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV, LeaveOneOut, PredefinedSplit
# import method to create the confusion matrix: 
from sklearn.metrics import confusion_matrix

from nonconformist.nc import NcFactory
from nonconformist.icp import IcpClassifier

In [2]:
#------------------------ ( GENERAL FUNCTIONS assignament 1 )!!!!! ------------------------

# dof function: --> calculate degrees of freedom
# ---------- PARAMETERS ---------------
# p = percentage of probability, that can be one of those values: [90,95,97,99]
# dof = the degrees of freedom (num_classes-1)
# -------------------------------------

def dof(p,dof):
    file_df = pd.read_csv(("./dof.csv"),sep=';', names=['df',90,95,97,99,999])
    thr = file_df[p].iloc[dof-1]
    return thr 


# mapping function: --> map a value into the correct interval
# ---------- PARAMETERS ---------------
# val = value to be mappe
# inter = list of all possible intervals
# -------------------------------------
def mapping(val,inter):
    for ls in inter:
        sx = ls[0]
        dx = ls[1]
        if (val >= sx and val <= dx): 
            return str(ls)
    # case in which the value is lower than the minimun interval
    if val < inter[0][0]:
        return str(inter[0])
    # case in which the value is higher than the maximum interval
    if val > inter[len(inter)-1][1]:
        return str(inter[len(inter)-1])
    


# define X2_merge function algorithm
# ---------- PARAMETERS ---------------
# dataset: dataframe of the dataset
# labels: label class
# continuity: array to identify the continuity features
# sigLevel: 90%,95%,97%,99% --> level of significance for the threashold. 
# dof: degrees of freedom (number_of_possible_classes - 1 ) ???
# globalDisc: is the dictionary in which you need to store possible intervals. 
# -------------------------------------

def chi2_thr(dataset, labels, continuity, sigLevel, degF, globalDisc): 
    
    print(continuity)
    
    # convert to dataframe: 
    df = pd.DataFrame(dataset)
    
    # take the threshold level for the X2 from the file
    thr = dof(sigLevel,degF)
    
    #take names of columns
    cols_name=(df.columns.values) # take all the names of attributes of your dataset. 
    
    #add labels
    #print(df.shape[1])
    df[df.shape[1]-1] = labels
    #print(df.shape[1])
    #add to continuity the true and false for the labels: 
    continuity.append(False)
    
    print(continuity)
    
    for idx in range(len(cols_name)):
    #for idx in range(len(continuity)-1): 
        
        i = cols_name[idx]  # take index name
        #print("*****", i, "*****")
        #check if the index correspond to a numerical attribute
        if continuity[i] == True: 
            
            # sort the entire dataframe 
            df = df.sort_values(i)
            
            # take label sorted
            label_sort = df[df.shape[1]-1].to_numpy()
            
            
            # take sort attribute 
            #arr_sort = sorted(pd.to_numeric((df[i]), downcast="float")) # creation of an array of values
            arr_sort = df[i].to_numpy()
            # cast to float
            arr_sort = arr_sort.astype(float)
            
            # remove the nan of that column and also in the labels in the same indexes
            [att_noNA, label_noNA] = (arr_sort[~np.isnan(arr_sort)], label_sort[~np.isnan(arr_sort)])                  
                              
            
            # Create contingency matrix and add the sum columns: 
            cont_matrix = pd.crosstab(att_noNA, label_noNA)
            cont_matrix['sum'] = cont_matrix[0]+cont_matrix[1]
            
            # take each pair of rows of contingency matrix, until n-1, for eache numerical attribute: 
            intervals = [[cont_matrix.index[i], cont_matrix.index[i+1]] for i in range(len(cont_matrix)-1)]
            
            # STEPS OF THE PROCESS: 
            print('-----------', i, '------------')
            
            # take all the lengths that you need
            l = len(cont_matrix) # length of conting martix  
            inter_len=len(intervals) # length of all intervals
            
            # choose a minimum number of intervals, to compress your numerical attribute. 
            max_compression = 30 # each numerical feature, will be compress to 20 possibile intervals. 
                       
            while(inter_len > max_compression):
                
                chi2 = [] # list of chi2 
                for k in range(l-1):

                    #calculate expected and observed, for each class of labels: 
                    b = cont_matrix.iloc[[k,k+1]] # conting matrix for the pair
                    #b = pd.crosstab(b,label_noNA)

                    tot_1= b[0].iloc[0]+b[0].iloc[1] # tot over the first col 
                    tot_2= b[1].iloc[0]+b[1].iloc[1] # tot over the second column
                    tot_sum = b['sum'].iloc[0]+b['sum'].iloc[1]  # tot over the 'sum' column

                    #first col elements 
                    obs_1 = b[0].iloc[0] 
                    obs_2 = b[0].iloc[1]

                    #second col elements 
                    obs_3 = b[1].iloc[0]
                    obs_4 = b[1].iloc[1]


                    # compute the expected and check if they are less than 0.5 or not. 
                    expected_1 = (b['sum'].iloc[0] * tot_1) / tot_sum # expected of the first element of the first row
                    if expected_1 < 0.5: 
                        expected_1 = 0.5 
                    expected_2 = (b['sum'].iloc[1] * tot_1) / tot_sum # expected of the second element of the first row
                    if expected_2 < 0.5: 
                        expected_2 = 0.5 

                    expected_3 = (b['sum'].iloc[0] * tot_2) / tot_sum # expected of the first element of the second row
                    if expected_3 < 0.5: 
                        expected_3 = 0.5 
                    expected_4 = (b['sum'].iloc[1] * tot_2) / tot_sum # expected of the second element of the second row
                    if expected_4 < 0.5: 
                        expected_4 = 0.5 


                    # compute the X2 
                    x2 = (((obs_1-expected_1)**2)/expected_1 ) + (((obs_2-expected_2)**2)/expected_2) + (((obs_3-expected_3)**2)/expected_3) + (((obs_4-expected_4)**2)/expected_4)
                    chi2.append(x2)
                    # add to the dictionary of chi2 of this attribute
                    # key of the dictionary is the index name of the numerical attribute 
                    # the value is the list of X2 for each pair of the clumns of that attribute
                    # chi2[i] = chi2
                   
                    
                # take the index of the minimum value chi2 on the list for each feature: 
                min_x2 = min(chi2)
                idx_min = chi2.index(min_x2)
                    
                # check if the value of the chi2 is less(or equal) than the threshold. 
                # if it is less, the H0 is not supported, so merge the two rows, otherwise let it unchanged.  
                if min_x2 <= thr: 
                    # remove the row after the index of the minimum: 
                    cont_matrix.drop(cont_matrix.index[idx_min+1], inplace=True)
                #recompute the len of intervals
                l = len(cont_matrix)
                
                #max_merge = max_merge - 1# update the merge total counter
                        # compute the last intervals : 
                intervals = [[cont_matrix.index[i], cont_matrix.index[i+1]] for i in range(len(cont_matrix)-1)]
                inter_len=len(intervals)
                # DEBUGGING 
                #print(inter_len)
 
            globalDisc[i] = intervals


# create object of feature transformer
# ---------- PARAMETERS ---------------
# Different parameters for each method (__init__, fit, transform)
# -------------------------------------

class FeatureTransformer: 
    # significance Level and degrees of freedom to default are 90 and 1. 
    def __init__(self, featureBag, discretization, missing_categorical, missing_numerical,sigLevel=90,degF=1): 
        self.featureBag = featureBag
        self.discretization = discretization
        self.missing_categorical = missing_categorical
        self.missing_numerical = missing_numerical
        
        # create and save the OHE (one hote encoder)
        self.OneHotEncoder = sk.preprocessing.OneHotEncoder(sparse=False)

        # all returning elements: 
        self.global_discret = {} # empty dictionary, to fill with chi2merge, for global discretizaion  
        self.local_discret = {} # empty dictionary, for local discretizzation. 
        self.mean = {} # list for all the means 
        self.var = {} # list for all variance 
        self.ftBag = []
        self.sigLevel = sigLevel # significance levele (90,95,97,99)
        self.degF = degF # degrees of freedoms
        
        # elements for missing categorical: 
        self.missing_cat_drop = []
        self.missing_cat_moda = {}
        self.missing_cat_random_unique = {}
        self.missing_cat_random_prob = {}
        self.missing_cat_random_uniform_unique = {}
        
        # elements for missing numerical: 
        self.missing_num_avg = {}
        self.missing_num_random_mean = {}
        self.missing_num_random_std = {}
        
        # list of dropped rows for missing categorical
        self.dropped_cat = []
        
        # list of dropped rows for missing numerical
        self.dropped_num = []
        
        
    def fit(self,x,y,continuity): 
        
        # set continuity 
        self.continuity = continuity       
        # remove all the nan
        #[arr_noNAN, label_noNAN] = (arr_x[~np.isnan(arr_x)], label_bool[~np.isnan(arr_x)])
        
        # ------------------------------------- FEATURE BAG ------------------------------------------
        
        if self.featureBag == True: 
            bool_arr = [True,False] # if the array is an arry full only of true i take all the columns. 
            #sample_arr = list(np.random.choice(bool_arr, x.shape[1])) # choice bunch of random features. 
            
            #for a better idea of bagging
            sample_arr = [False for _ in range(x.shape[1])]
            for _ in range(x.shape[1]): 
                sample_arr[random.randint(0,x.shape[1]-1)]=True
            """
            v = np.where(sample_arr == True)
            x = x[:, v] # take only columns with index equal to true 
            """
            self.ftBag = sample_arr
            
            #return self # if you run the self here all the following code is not correct because update only here. 
            v = [i for i, x in enumerate(self.ftBag) if x]
            x = x[v] 

         
        if self.featureBag == False:
            # do anithing
            self.ftBag = x
            #return self
               
        
        
        
        # ------------------------------------- MISSING CATEGORICAL ------------------------------------------
        if self.missing_categorical == 'drop':
            # remove on the transform ( drop the rows containing nan values --> task computed by transformation )           
            # save all rows to drop due to categorical rows with NaN
            
            """
            ls_na = []
            
            # --- do directly on the transform ( drop the rows containing nan values --> task computed by transformation ) ---
            cols_name=(x.columns.values) # take all the names of attributes of your dataset. 
            for idx in range(len(cols_name)): 
                #take correct index 
                i = cols_name[idx]
                if continuity[i] == False:
                    
                    # take indexes of all nan values in current categorical column 
                    rows_with_nan = (x[pd.isna(x[i])]).index
                    
                    # convert indexes found to list
                    index_rows_na = (rows_with_nan.values).tolist()
                    
                    # concat to the full list: 
                    ls_na = ls_na + index_rows_na
            
            # save the list of all rows:
            self.missing_cat_drop = ls_na
            """
            
            
        
        if self.missing_categorical == 'most_frequent':
            cols_name=(x.columns.values) # take all the names of attributes of your dataset. 
            for idx in range(len(cols_name)): 
                #take correct index 
                i = cols_name[idx]
                if continuity[i] == False: # check for columns that are categorical
                    # take unique values and counting of each values
                    mf, mf_count = np.unique((x[i]).dropna(), return_counts=True)
                    # take the highe value: 
                    max_count = max(mf_count)
                    # convert to list: 
                    mf_count = mf_count.tolist()
                    # take the most frequent element by its index: 
                    moda = mf[mf_count.index(max_count)]
                    self.missing_cat_moda[i] = moda
           
            
        if self.missing_categorical == 'random':
            # use "np.random.coiche" with parameters: size = 1, p = vector_prob
            # vectori_prob is a vector containing the probability of each item inside the feature
            # probability of each item is given by: count(item_occurrences)/len(feature).
            # create lambda object to divide occurrences of each value of your categorical vector.
            # in this way you can find the probability of each value. 
            
            # create object/function to divide each element
            # you need to round,the probability to the first decimal place, in way to obtain probability sum = 1 
            # np.around(,decimals=1)
            divide = (lambda v: ((v/l_s1))) 
            
            
        
            # minus -1 on the loop because we removed the label column. 
            cols_name=(x.columns.values) # take all the names of attributes of your dataset. 
            for idx in range(len(cols_name)): 
                #take correct index 
                i = cols_name[idx]
                if continuity[i] == False: # check for columns that are categorical
                    # take the len of the feature vector: 
                    l_s1=len(x[i])
                    
                    # take each element of the coulmn uniquely(s1) removing NaN values, 
                    # and the number of occurrences of each element
                    s1, s1_count = np.unique((x[i]).dropna(), return_counts=True)
                    
                    # divide each item by the total number of occurrences: 
                    vector_prob = divide((s1_count))
                    
                    # compute some check, to be correct: 
                    if (len(vector_prob) < len(s1)): 
                        #remove an element (the element with the smallest probability ) from bucket:
                        idx = np.where(vector_prob == min(vector_prob))
                        s1 = np.delete(s1, idx)
                     
                    if (len(vector_prob) > len(s1)):
                        # remove an element from prob: 
                        m = min(vector_prob)
                        vector_prob = np.delete(vector_prob, np.where(vector_prob == m))                  
                    
                    
                    #Save all possible unique values and also vector of probabilities: 
                    self.missing_cat_random_unique[i] = s1
                    # when you save the probability occurrence of each element, you need to round 
                    # the probability to the first decimal place, in way to obtain probability sum = 1 
                    self.missing_cat_random_prob[i] = vector_prob

                    
            
        if self.missing_categorical == 'random_uniform':
            # random_uniform = each value has the same probability to came out. 
            # (for example: with 10 values, each value has 1/10 probabaility.)
            
            # Use np.random.choice() --> with parameters: array, size = 1 
            cols_name=(x.columns.values) # take all the names of attributes of your dataset. 
            for idx in range(len(cols_name)): 
                #take correct index 
                i = cols_name[idx]
                if continuity[i] == False:
                    s1 = np.unique((x[i]).dropna())
                    self.missing_cat_random_uniform_unique[i] = s1
                    
        
        
        
        
        
        # ------------------------------------- MISSING NUMERICAL --------------------------------------------
        if self.missing_numerical == 'drop':
            # same of missing categorical, but compute directly on the transform. 
            pass
            
            
        if self.missing_numerical == 'avarage':
            cols_name=(x.columns.values) # take all the names of attributes of your dataset. 
            for idx in range(len(cols_name)): 
                #take correct index 
                i = cols_name[idx]
                if continuity[i] == True:
                    cl = x[i]
                    tmp_me = np.mean(cl.dropna().to_numpy().astype(float)) 
                    self.missing_num_avg[i] = tmp_me
            
        if self.missing_numerical == 'random':
            # use as function 'np.random.normal()--> loc = mean, scale = std, size = 1'
            cols_name=(x.columns.values) # take all the names of attributes of your dataset. 
            for idx in range(len(cols_name)): 
                #take correct index 
                i = cols_name[idx]
                if continuity[i] == True:
                    cl = x[i]
                    tmp_me = np.mean(cl.dropna().to_numpy().astype(float)) 
                    tmp_std = np.std(cl.dropna().to_numpy().astype(float))
                    self.missing_num_random_mean[i] = tmp_me
                    self.missing_num_random_std[i] = tmp_std
            
        if self.missing_numerical == 'random_uniform':
            # 'np.random.uniform() --> parameters: low = min, high = max, size = how_many_element'
            pass
       
    
        # ------------------------------------- DISCRETIZATION ------------------------------------------
                                
        if self.discretization == 'global': 
            # only numerical attributes (feature) must be converted into a set of attribute 
            # with chi2global function. 
            
            # call the chi2merge algo: 
            chi2_thr(x, y, continuity, self.sigLevel, self.degF, self.global_discret)
            
            # mapping data from your numerical attribute, to your dictionary data    
            """
            cols_name=(x.columns.values) # take all the names of attributes of your dataset. 
            for idx in range(len(cols_name)): 
                #take correct index 
                i = cols_name[idx]
            
                if continuity[i] == True:
                    # take the set of inverlas for that numerical feature
                    dic = self.global_discret.get(i)
                    # map all the rows
                    new_vals = []
                    for j in range(df_na[i].shape[0]):
                        # convert into float the current val
                        v = float(df_na[i].iloc[j])
                        # map the value to the correct interval
                        r = mapping(v,dic)
                        #append it to the list
                        new_vals.append(r) 

                    # substitute vals with intervals:
                    x[i] = new_vals        
            
            #x = x.iloc[:,:-1]
            
            #print(type(x[1]))      
            # fit the one hot encoding with the coresponding dataset without labels:         
            """
            
        
        if self.discretization == 'local': 
            # take the numerical attributes (feature). For each take unique values without duplicates. sort it.
            cols_name=(x.columns.values) # take all the names of attributes of your dataset. 
            for idx in range(len(cols_name)): 
                #take correct index 
                i = cols_name[idx]
                
                if continuity[i] == True:
                    df_disc = x[i].replace('?', np.nan) 
                    df_i_disc_unic = (df_disc.unique()).astype(float)
                    srt = np.sort(df_i_disc_unic) # srt is an np.array object. 
                    
                    # add sorted list to the dictionary of local discretization: 
                    self.local_discret[i] = srt
            
        
        if self.discretization == 'scaled': 
            # mean normalizzation (perform mean and standard deviation for each numerical attribute)            
            cols_name=(x.columns.values) # take all the names of attributes of your dataset. 
            for idx in range(len(cols_name)): 
                #take correct index 
                i = cols_name[idx]
                if continuity[i] == True:
                    # compute mean and standard deviation for each row
                    row = x[i] # take the column  
                    me = np.mean(row.dropna().to_numpy().astype(float))
                    var = np.std(row.dropna().to_numpy().astype(float))
                    #append all the mean and all the var
                    self.mean[i]=me
                    self.var[i]=var
                    # append to the self variable
                    
                
            
        
        if self.discretization == 'None': 
            x=x
             

       # After all preprocessing of fitting, compute fitting of you ine hot encoder 
        #self.OneHotEncoder.fit(x)          
                
        # return dataset and label:
        # return arr_x, label_bool
        
        return self # to update the dates.

        
    def transform(self,x):   
        """
            self.x = x # the dataframe 
            self.y = y # the label array

            # remove the last column of 
            x = x.iloc[:, :-1]
            # convert dataframe into numpy array. 
            arr_x = x.to_numpy()

            le = sk.preprocessing.LabelEncoder() # encode labels and convert it into numpy array. 
            label_bool = le.fit_transform(y)# fit and convert automatically in numpy array.
            
        """ 
        
         # ------------ Features Bag: ------------------------
        
        if self.featureBag == True: 
            #x = x.iloc[:, :-1]#remove label
            #v = np.where(self.ftBag == True)
            v = [i for i, x in enumerate(self.ftBag) if x]
            x = x[v] 
                      
        
        elif self.featureBag == False: 
            x = x 
            

        # ------------ Missing Categorical: ------------------------
        
        if self.missing_categorical == 'drop':
            #print("ciao sono entrato")
            # simply drop and inplace, the list of categorical rows with NaN
                        # do directly on the transform ( drop the rows containing nan values --> task computed by transformation )
            cols_name=(x.columns.values) # take all the names of attributes of your dataset. 
            for idx in range(len(cols_name)): 
                #take correct index 
                i = cols_name[idx]
                if self.continuity[i] == False:
                    
                    # take indexes of all nan values in current categorical column 
                    rows_with_nan = (x[pd.isna(x[i])]).index
                    
                    # convert indexes found to list
                    index_rows_na = (rows_with_nan.values).tolist()
                    
                    # append to list of dropped rows: 
                    self.dropped_cat.append(index_rows_na)
                    
                    x.drop(index_rows_na, inplace=True)
                    
            
        if self.missing_categorical == 'most_frequent':
            cols_name=(x.columns.values) # take all the names of attributes of your dataset. 
            for idx in range(len(cols_name)): 
                #take correct index 
                i = cols_name[idx]
                if self.continuity[i] == False:
                    moda = self.missing_cat_moda.get(i)
                    # replace NaN with moda 
                    x[i] = x[i].apply(lambda x: ( moda if x is np.nan else  x))
            
           
            
        if self.missing_categorical == 'random':
            # use "np.random.coiche" with parameters: size = 1, p = vector_prob
            # vectori_prob is a vector containing the probability of each item inside the feature
            # probability of each item is given by: count(item_occurrences)/len(feature).
            # create lambda object to divide occurrences of each value of your categorical vector.
            # in this way you can find the probability of each value. 
            
            def rnd_choice(i):
                prob = FT.missing_cat_random_prob.get(i)
                bucket = FT.missing_cat_random_unique.get(i)
                # Normalizzation of probability vector. 
                prob /= prob.sum()
                el_random = np.random.choice(bucket, size=1, p=prob)
                element = el_random[0]
                return element

            cols_name=(x.columns.values) # take all the names of attributes of your dataset. 
            for idx in range(len(cols_name)): 
                #take correct index 
                i = cols_name[idx]
                if self.continuity[i] == False:
                    # take randomly one oblect, with random choice, according to the distribution:                     
                    # replace with random value 
                    x[i] = x[i].apply(lambda x: ( rnd_choice(i) if x is np.nan else  x))

                    
            
        if self.missing_categorical == 'random_uniform':
            # random_uniform = each value has the same probability to came out. 
            # (for example: with 10 values, each value has 1/10 probabaility.)
            # Use np.random.choice() --> with parameters: array, size = 1 
            
            def rnd_uni_choice(i):
                el = np.random.choice(self.missing_cat_random_uniform_unique.get(i),1)
                return el[0]
            
            cols_name=(x.columns.values) # take all the names of attributes of your dataset. 
            for idx in range(len(cols_name)): 
                #take correct index 
                i = cols_name[idx]
                if self.continuity[i] == False:
                    x[i] = x[i].apply(lambda x: ( rnd_uni_choice(i) if x is np.nan else  x))
        
        
        
        # ------------ Missing Numerical: ------------------------
        if self.missing_numerical == 'drop':
            #print("ciao sono entrato")
            # I NEED TO DROP NUMERICAL HERE BECAUSE BEFORE I DROP SOME ROWS WIT CATEGORICAL NAN. 
            # TO AVOID INCONGRUENCES I NEED TO REMOVE NUMERICAL NAN AFTER I'VE REMOVED THE CATEGORICAL NAN'S.
            
            # remove on the transform ( drop the rows containing nan values --> task computed by transformation )           
            # save all rows to drop due to NUMERICAL with NaN
            ls_na = []
            
            # do directly on the transform ( drop the rows containing nan values --> task computed by transformation )
            cols_name=(x.columns.values) # take all the names of attributes of your dataset. 
            for idx in range(len(cols_name)): 
                #take correct index 
                i = cols_name[idx]
                if self.continuity[i] == True:
                    
                    # take indexes of all nan values in current categorical column 
                    rows_with_nan = (x[pd.isna(x[i])]).index
                    
                    # convert indexes found to list
                    index_rows_na = (rows_with_nan.values).tolist()
                    
                    # append dropped rows to list of numerical dropped rows
                    self.dropped_num.append(index_rows_na)
                    
                    x.drop(index_rows_na, inplace=True)
                    
                    
        if self.missing_numerical == 'avarage':
            cols_name=(x.columns.values) # take all the names of attributes of your dataset. 
            for idx in range(len(cols_name)): 
                #take correct index 
                i = cols_name[idx]
                if self.continuity[i] == True:
                    avg = self.missing_num_avg.get(i)
                    # replace NaN with moda 
                    x[i] = x[i].apply(lambda x: ( avg if x is np.nan else  x))
            
            
        if self.missing_numerical == 'random':
            # use as function 'np.random.normal()--> loc = mean, scale = std, size = 1'
            def rand_num(i): 
                me = self.missing_num_random_mean.get(i)
                st = self.missing_num_random_std.get(i)
                element = np.random.normal(me, st, 1)
                return element
            
            cols_name=(x.columns.values) # take all the names of attributes of your dataset. 
            for idx in range(len(cols_name)): 
                #take correct index 
                i = cols_name[idx]
                if self.continuity[i] == True:
                    x[i] = x[i].apply(lambda x: ( rand_num(i) if x is np.nan else  x))
            
        if self.missing_numerical == 'random_uniform':
            # 'np.random.uniform() --> parameters: low = min, high = max, size = how_many_element'
            cols_name=(x.columns.values) # take all the names of attributes of your dataset. 
            for idx in range(len(cols_name)): 
                #take correct index 
                i = cols_name[idx]
                if self.continuity[i] == True:
                    l = x[i].astype(float).min()
                    h = x[i].astype(float).max()
                    x[i] = x[i].apply(lambda x: ( np.random.uniform(l,h,1) if x is np.nan else  x))
            
        # try to print dataset
        #print(x.head)
        #print("–-------------------------")  
        # check if there is nans 
        #print(x[x.isna().any(axis=1)])
        
        # return dataset and label:
        # return arr_x, label_bool  
        
        # ------------ Discretizations: ------------------------ 
        
        if self.discretization == 'global':
            
            # only numerical attributes (feature) must be converted into a set of attribute 
            # with chi2global function. 
            
            # Compute fit and transform of one hot encoding: 
            # x = self.OneHotEncoder.fit_transform(x)
            #x = self.OneHotEncoder.transform(x)
            cols_name=(x.columns.values) # take all the names of attributes of your dataset. 
            for idx in range(len(cols_name)): 
                #take correct index 
                i = cols_name[idx]
            
                if self.continuity[i] == True:
                    # take the set of inverlas for that numerical feature
                    dic = self.global_discret.get(i)
                    # map all the rows
                    print(i)
                    print("------------------------------")
                    new_vals = []
                    for j in range(x[i].shape[0]):
                        # convert into float the current val
                        v = float(x[i].iloc[j])
                        # map the value to the correct interval
                        r = mapping(v,dic)
                        #print(r)
                        if r == None:
                            print("----------------------------")
                            print("il valore che non riesco ad inserire è: ", v)
                            print("gli intervalli possibili sono: ", dic)
                            print("----------------------------")
                        #append it to the list
                        new_vals.append(r) 

                    # substitute vals with intervals:
                    x[i] = new_vals 
                   
            

            
        
        if self.discretization == 'local': 
            # the numerical attributes (feature) take unique values without duplicates and sort it. 
            # ( you should transform into float )
            
            def check_local(arr,n): 
                vector = np.vectorize(np.float)
                arr = vector(arr)
                res = ((np.array(arr))<= float(n))
                return res.astype(int)
                                                   
            
            cols_name=(x.columns.values)#take all the cols names 
            for i in range(len(cols_name)): 
                #print(vals[i])
                idx = cols_name[i] # index name
                if self.continuity[idx] == True:
                    # take the unique and sorted array: 
                    arr_sort = self.local_discret[idx]
                    # apply digitize of numpy in order to binarize the vector: 
                    x[idx] = x[idx].apply(lambda el: check_local((arr_sort),el))
            
        
        if self.discretization == 'scaled': 
            # substitute eache value of the numerical feature, with the normalized values. 
            # x' = x-median/var 
            
            cols_name=(x.columns.values)#take all the cols names 
            for i in range(len(cols_name)): 
                #print(vals[i])
                idx = cols_name[i] # index name
                if self.continuity[idx] == True:
                    me = self.mean.get(idx)
                    var = self.var.get(idx)
                    x[idx] = x[idx].apply(lambda x: ((float(x)-me)/var) if x!='nan' else x)
            
            
            
        
        if self.discretization == 'None': 
            x=x 

        
            
        return x # return transformed data     

    
# DEFINE A CLASS TO OHE THE RESULT OF YOUR FEATURE-TRANSFORMER:
# ------------------------------
# --- INIT PARAMETERS ---
# typeDisc: type of discretization for the OHE 
# dataFit: dataset with which you should fit your OHE encoder
# dataTr: datata that you want to OHE dased on type and dataFit
# --- OUTPUTS ---> OHE of dataTR based on type of discrretization and fitted dataset.
# ------------------------------
class OHEencoding:
    def __init__(self, typeDisc, dataFit, dataTr, continuity):
        self.typeDisc = typeDisc
        self.dataFit = dataFit
        self.dataTr = dataTr
        self.continuity = continuity
        # create your OHE encoder
        self.OneHotEncoder = sk.preprocessing.OneHotEncoder(sparse = False)
        self.cols = []
        

    def fitTransform(self): 
        
        if self.typeDisc == 'global':
            self.OneHotEncoder.fit(self.dataFit.astype(str))
            res_global = self.OneHotEncoder.transform(self.dataTr.astype(str))
            
            return res_global

        if self.typeDisc == 'local': 
            res_local = [] # create an empty list, that will corresponds to the resulting row from the dataset: 
            cols_name=(self.dataFit.columns.values)#take all the cols names 
            for i in range(len(cols_name)): 
                #print(vals[i])
                idx = cols_name[i] # index name
                if self.continuity[idx] == False:
                    self.cols.append(idx)
                    ts = np.array(np.unique(self.dataFit[idx]))
                    self.OneHotEncoder.fit(ts.reshape(-1,1))
                    result_test = self.OneHotEncoder.transform(np.array(self.dataTr[idx]).reshape(-1,1)).tolist()
                    #print(result_test[0])
                    #print(type(result_test[0]))
                    res_local.append(result_test[0])
        
                    #res[i]
                if self.continuity[idx] == True:  
                    tmp = self.dataTr[idx].tolist() 
                    tmp_list = tmp[0]#.tolist()
                    res_local.append(tmp_list)
                    #ts = np.array(self.dataFit[idx])
                    #self.OneHotEncoder.fit(ts.reshape(-1,1))
                    #tmp = self.OneHotEncoder.transform(np.array(self.dataTr[idx]).reshape(-1,1))
                    #self.dataTr[idx] = list(tmp)
            return res_local   
        
             
            #res = self.dataTr[idx]
            #self.OneHotEncoder.fit(self.dataFit[self.cols].astype(str))
            #res = self.OneHotEncoder.transform(self.dataTr[self.cols].astype(str))                   
        else:   
            res_all = [] # create an empty list, that will corresponds to the resulting row from the dataset: 
            cols_name=(self.dataFit.columns.values)#take all the cols names 
            for i in range(len(cols_name)): 
                #print(vals[i])
                idx = cols_name[i] # index name
                if self.continuity[idx] == False:
                    self.cols.append(idx)
                    ts = np.array(np.unique(self.dataFit[idx]))
                    self.OneHotEncoder.fit(ts.reshape(-1,1))
                    result_test = self.OneHotEncoder.transform(np.array(self.dataTr[idx]).reshape(-1,1)).tolist()
                    #print(result_test[0])
                    #print(type(result_test[0]))
                    res_all.append(result_test[0])


                    #res[i]
                if self.continuity[idx] == True:  
                    tmp = self.dataTr[idx]
                    #print(type(tmp[0]))
                    #print(tmp[0])
                    res_all.append(tmp[0])
            
            return res_all

        
    # create your OHE encorder: 
    # fit your OHE with dataFit
    # convert you OHE encoder with dataTr 

In [3]:
# ------------------- GENERAL FUNCTIONS FROM ASSIGNAMENT 2 --------------------
# ----------------------------ATTRIBUTES TO NUMBERS----------------------------
# Description = from a dataset of categorical attributes, convert into integer. 
# Input = dataset (all attributes/features must be categorical)
# Output = dataset with all attributes rows converted into integer. 
# -----------------------------------------------------------------------------
def attributeToNum(dataset):
    # take all attributes of the dataset: 
    attributes = dataset.columns.values
    # define the encoder: 
    label_enc = sk.preprocessing.LabelEncoder()
    # loop over all the columns: 
    for i in attributes:
        # take the column (attributes) casted to string
        new_att = dataset[i].astype(str) 
        # apply the fit transform encoding
        new_att_transformed = label_enc.fit_transform(new_att)
        # substitute column of dataset with new column transformed: 
        dataset.loc[:,[i]]=[new_att_transformed]
        

    # retrun transformed dataset: 
    return dataset



# ------------------- ATTRIBUTES TO NUMBERS OBJECT CLASS ----------------------
# -----------------------------------------------------------------------------
# Obecject class to convert data into all integer number based on labelEncoder of feat. 
# - use method "convert", to convert a new instance
class attToNumClass: 
    def __init__(self, dataset):
        # Create labelEncoder to use 
        self.label_enc = sk.preprocessing.LabelEncoder() 
        self.dataset = dataset
        
    def convert(self, new_instance): 
        # take the columns 
        attributes = new_instance.columns.values
        # loop over columns:
        for i in attributes:
            # take column of new instance and column of dataset. 
            # for the new attribute: 
            att_new_inst = new_instance[i].astype(str)
            # for the dataset: 
            att_data = self.dataset[i].astype(str)
            # fit self.label_enc on the self.dataset
            self.label_enc.fit(att_data)
            # transform the new_instance column using fitted LabelEncoder
            att_new_inst_transformed = self.label_enc.transform(att_new_inst)
            
            # substitute column of dataset with new column transformed: 
            new_instance[i] = att_new_inst_transformed
        # return new converted dataset. 
        return new_instance
# -----------------------------------------------------------------------------

# -----------------------------------------------------------------------------
# --- allignDrop FUNCTION ---
# Description: in case you use Drop in some where on Feature transformer, allign labels length
# INPUT: FT --> FeatureTransformer object, y --> original label list. 
# OUTPUT: --> new label list, without dropped instances
# -----------------------------------------------------------------------------
def allignDrop(FT,y):
    first_list = FT.dropped_num
    second_list = FT.dropped_cat

    # flat the two list: 
    flat_first_list = [item for sublist in first_list for item in sublist]
    flat_second_list = [item for sublist in second_list for item in sublist]

    # concatenate the two flatten lists: 
    total_to_remove = flat_first_list + flat_second_list
    
    # remove from the labels list: 
    new_y = np.delete(y, total_to_remove)
    
    return new_y

# -----------------------------------------------------------------------------
    

# ---------------- BAG SPLIT GENERIC FUNCTION --------------------
# generic function to split data in a bagging way. 
def bag_split(x,y): 
    n = len(y)
    a= np.array(range(0,n))
    choose = np.zeros(n, dtype=bool)
    for _ in range(0,n):
        choose[np.random.choice(a)] = True
    return x[choose], x[~choose], y[choose], y[~choose]


# ----------- SELF OPTIMIZING CLASSIFIER OBJECT ------------------
class selfOptimizingClassifier: 
    # define the input statements of our object, to decide the optimizzation: 
    def __init__(self, classifier, params_grid = None, k_opt = 5, calibration = 0.2, compute_proba = 'model'): 
        
        # assign 
        self.classifier = classifier
        self.params_grid = params_grid
        self.params_win = None
        #check if kopt is a positive integer
        self.k_opt  = k_opt
        
        self.model = None
        self.conf_matrix = None # the object "cm" request by the assignament. 
        self.SumMatrix = None
        self.acc_conf = None
        self.calibrated_model = None 
        self.calibration = calibration
        self.compute_proba = compute_proba
        #in order to do compute_proba=model we need to store the whole dataset minus xc---> so xt
        self.xt=None
        
        
    #optimize: the best parameters in params grid for classifier are chosen via GridSearch according to 
    #k opt using (Xt', Yt'), such best parameters are stored in params win dictionary.
    def __optimize(self,xt,yt):
        #find opt params win
        clf = None
        if self.k_opt == 0: #if k=0, simply apply a split in train and test
            ps = sk.model_selection.PredefinedSplit(np.random.choice[-1,0], size=len(yt))#bagging if k=0
            ps.get_n_splits()
            clf = GridSearchCV( estimator = self.classifier, param_grid = self.params_grid, cv=ps, n_jobs=2)
        if self.k_opt == 1:
            clf = GridSearchCV( estimator = self.classifier, param_grid = self.params_grid, cv=LeaveOneOut(), n_jobs=2)
        if self.k_opt >=2:
            clf = GridSearchCV( estimator = self.classifier, param_grid = self.params_grid, cv=self.k_opt, n_jobs=2)


        clf.fit(xt,yt)

        return clf.best_params_


    #if calibrate != 0 a conformal prediction model is build by calibrating model on (Xc, Yc),
    #such a model is stored in an attribute called calibrated model

    # compute conformal prediction. 
    def __calibrate(self,xc,yc):
        nc_dt = NcFactory.create_nc(self.model)
        icp = IcpClassifier(nc_dt)
        icp.calibrate(xc,yc)
        return icp

    def predict(self,x):
        if (x is not None) and len(x) !=0:
            try:
                return self.model.predict(x)
            except:
                return np.nan
        return np.nan

#The predict proba method will return the probability returned by the model if compute proba = 'model'
#the array of probabilities given by the model is returned, otherwise it is returned the (normalized) column
#corresponding to the prediction of the confusion matrix computed at step (b).
    def predict_proba(self,x):
        if self.compute_proba != 'model' and self.compute_proba !=0:
            if self.conf_matrix is not None:
                #probabilities returned according to confusion matrix computed
                try:
                    proba = [(self.conf_matrix[:,int(pred)])/self.conf_matrix[:,int(pred)].sum() for pred in self.model.predict(x)]
                    return np.array(proba)                
                except:
                    return np.nan
            return np.nan
        else:
            if self.compute_proba == 'model':
                arr_prob= self.model.predict_proba(self.xt)
                return np.array(arr_prob)

    # eps (epsilon) correspond to the significative of conformal prediction
    def conformal_predict(self,x,eps=0.1):
        if self.calibration != 0 :
            try:
                return self.calibrated_model.predict(x,eps)
            except:
                return np.nan
        print("Not feasible calibration = 0")
        return np.nan

    #perform accuracy given the confusion matrix
    def accuracy(self):
        if self.compute_proba != 0 and self.compute_proba != 'model':
            if self.conf_matrix is not None:
                return self.conf_matrix.diagonal().sum() / self.SumMatrix
        return np.nan
    
        
    # x = dataset, y = labels
    def fitData(self,x,y):
        xd = x
        yd = y
        xt,yt = None, None
        xc,yc = None, None
        xp,yp = None, None
        
        # Divide in xt and yt
        if self.calibration != 0: 
            if self.calibration == 'bagging': 
                print("calibration, associated to bagging")
                xt, xc, yt, yc = bag_split(xd, yd)
                
            elif self.calibration > 0 and self.calibration < 1: # it is a value between 0 and 1. 
                xt, xc, yt, yc = train_test_split(xd, yd, test_size = self.calibration)
        else:
            # if you arrive here, calibration set is 0 and xt and yt remain equal. 
            if self.calibation==0:
                xt=xd
                yt=yd
                
        
   

        # Divide in x't and y't
        # recompute splitting to create  x't and y't and also xp and yp (xp and yp are used to create confusion matrix)
        if self.compute_proba != 'model':
            if self.compute_proba == 'bagging':
                print("probability set extracted with bagging")
                xt, xp, yt, yp = bag_split(xd, yd)
            else:
                xt, xp, yt, yp = train_test_split(xd, yd, test_size=self.compute_proba)
                
        #control check for assign the parameters of compute_proba
        #in this case xt because it represent x-xc
        if self.compute_proba == 'model' and self.calibration !=0:
            self.xt=xt
            

        
        #Retrieve the best parameters 
        self.params_win = self.__optimize(xt,yt)# best parameter are obtain by call optimize method and assign it to self.params_win
        
        # set the classifier with winning parameters: 
        self.classifier.set_params(**self.params_win)#with ** mean that params_win has a dictionary structure
        
        self.classifier.get_params(deep=True)
        # classifier is fitted with params_win on (Xt,Yt) and saved on a object attribute model, 
        # represent the model obatin from the classifier
        self.model = self.classifier.fit(xt,yt)
        
        # compute the confusion matrix
        if self.compute_proba != 'model' and self.compute_proba != 0:
            # compute the prediction by the classifier. 
            y_pred = self.model.predict(xp)
            # compute and save confusion matrix 
            self.conf_matrix = confusion_matrix(yp, y_pred)
            
            # we need now to normilize our confuzion matrix. 
            # ??? sum_matrix ??? 
            self.SumMatrix = self.conf_matrix.sum()
            self.acc_conf = self.accuracy()
            
        #if self.compute_proba=='model':
         #   print(self.xt)
          #  self.compute_proba(self.xt)
    
        #if calibrate != 0 the method calibrate is called
        if  self.calibration != 0 :
            self.calibrated_model = self.__calibrate(xc,yc)
        
        return self
"""
class selfOptimizingClassifier: 
    # define the input statements of our object, to decide the optimizzation: 
    def __init__(self, classifier, params_grid = None, k_opt = 5, calibration = 0.2, compute_proba = 'model'): 
        
        # assign 
        self.classifier = classifier
        self.params_grid = params_grid
        self.params_win = None
        #check if kopt is a positive integer
        self.k_opt  = k_opt
        
        self.model = None
        self.conf_matrix = None # the object "cm" request by the assignament. 
        self.calibrated_model = None 
        self.calibration = calibration
        self.compute_proba = compute_proba
        
    #optimize: the best parameters in params grid for classifier are chosen via GridSearch according to 
    #k opt using (Xt', Yt'), such best parameters are stored in params win dictionary.
    def __optimize(self,xt,yt):
        #find opt params win
        clf = None
        if self.k_opt == 0: #if k=0, simply apply a split in train and test
            ps = sk.model_selection.PredefinedSplit(np.random.choice[-1,0], size=len(yt))#bagging if k=0
            ps.get_n_splits()
            clf = GridSearchCV( estimator = self.classifier, param_grid = self.params_grid, cv=ps, n_jobs=2)
        if self.k_opt == 1:
            clf = GridSearchCV( estimator = self.classifier, param_grid = self.params_grid, cv=LeaveOneOut(), n_jobs=2)
        if self.k_opt >=2:
            clf = GridSearchCV( estimator = self.classifier, param_grid = self.params_grid, cv=self.k_opt, n_jobs=2)

        # compute fitting here with parameters: 
        #print(type(xt), type(yt))
        #xt = np.array(xt) 
        #yt = np.array(yt)
        #print(type(xt), type(yt))

        clf.fit(xt,yt)

        return clf.best_params_


    #if calibrate != 0 a conformal prediction model is build by calibrating model on (Xc, Yc),
    #such a model is stored in an attribute called calibrated model

    # compute conformal prediction. 
    def __calibrate(self,xc,yc):
        nc_dt = NcFactory.create_nc(self.model)
        icp = IcpClassifier(nc_dt)
        icp.calibrate(xc,yc)
        return icp

    def predict(self,x):
        if (x is not None) and len(x) !=0:
            try:
                return self.model.predict(x)
            except:
                return np.nan
        return np.nan

#The predict proba method will return the probability returned by the model if compute proba = 'model'
#the array of probabilities given by the model is returned, otherwise it is returned the (normalized) column
#corresponding to the prediction of the confusion matrix computed at step (b).
    def predict_proba(self,x):
        if self.compute_proba != 'model' and self.compute_proba !=0:
            if self.conf_matrix is not None:
                #probabilities returned according to confusion matrix computed
                try:
                    proba = [(self.conf_matrix[:,pred])/self.conf_matrix[:,pred].sum() for pred in self.model.predict(x)]
                    return np.array(proba)                
                except:
                    return np.nan
            return np.nan
        else:
            if self.compute_proba == 'model':
                #i pass xt because it is: x-xc
                y_pred_model=self.model.predict(self.xt) ### !!!!!!!! this are the predictions, NO PROB ARRAY
                return np.array(y_pred_model)

    # eps (epsilon) correspondo to the significative of conformal prediction
    def conformal_predict(self,x,eps=0.1):
        if self.calibration != 0 :
            try:
                return self.calibrated_model.predict(x,eps)
            except:
                return np.nan
        print("Not feasible calibration = 0")
        return np.nan


    def accuracy(self):
        if self.compute_proba != 0 and self.compute_proba != 'model':
            if self.conf_matrix is not None:
                return self.conf_matrix.diagonal().sum() / self.sum_matrix
        return np.nan
        
    # x = dataset, y = labels
    def fitData(self,x,y):
        xd = x
        yd = y
        xt,yt = None, None
        xc,yc = None, None
        xp,yp = None, None
        
        # Divide in xt and yt
        if self.calibration != 0: 
            if self.calibration == 'bagging': 
                print("calibration, associated to bagging")
                xt, xc, yt, yc = bag_split(xd, yd)
                
            elif self.calibration > 0 and self.calibration < 1: # it is a value between 0 and 1. 
                xt, xc, yt, yc = train_test_split(xd, yd, test_size = self.calibration)
        
        # if you arrive here, calibration set is 0 and xt and yt remain equal. 

        # Divide in x't and y't
        # recompute splitting to create  x't and y't and also xp and yp (xp and yp are used to create confusion matrix)
        if self.compute_proba != 'model':
            if self.compute_proba == 'bagging':
                print("probability set extracted with bagging")
                xt, xp, yt, yp = bag_split(xd, yd)
            else:
                xt, xp, yt, yp = train_test_split(xd, yd, test_size=self.compute_proba)
        '''             
        # prepare input data
        def prepare_inputs(xt, X_test):
            oe = OrdinalEncoder()
            oe.fit(X_train)
            X_train_enc = oe.transform(X_train)
            X_test_enc = oe.transform(X_test)
        return X_train_enc, X_test_enc

        # prepare target
        def prepare_targets(y_train, y_test):
            le = LabelEncoder()
            le.fit(y_train)
            y_train_enc = le.transform(y_train)
            y_test_enc = le.transform(y_test)
        return y_train_enc, y_test_enc
                
        
        X_train_enc, X_test_enc = prepare_inputs(X_train, X_test)
        # prepare output data
        y_train_enc, y_test_enc = prepare_targets(y_train, y_test)
        '''  
        
        #Retrieve the best parameters 
        self.params_win = self.__optimize(xt,yt)# best parameter are obtain by call optimize method and assign it to self.params_win
        
        # set the classifier with winning parameters: 
        self.classifier.set_params(**self.params_win)#with ** mean that params_win has a dictionary structure
        
        # classifier is fitted with params_win on (Xt,Yt) and saved on a object attribute model, 
        # represent the model obatin from the classifier
        self.model = self.classifier.fit(xt,yt)
        
        # compute the confusion matrix
        if self.compute_proba != 'model' and self.compute_proba != 0:
            # compute the prediction by the classifier. 
            y_pred = self.model.predict(xp)
            # compute and save confusion matrix 
            self.conf_matrix = confusion_matrix(yp, y_pred)
            # we need now to normilize our confuzion matrix. 
            # ??? sum_matrix ??? 
            # self.sumMatrix = self.conf_matrix.sum().sum()
        
        #if calibrate != 0 the method calibrate is called
        if  self.calibration != 0 :
            self.calibrated_model = self.__calibrate(xc,yc)
"""

'\nclass selfOptimizingClassifier: \n    # define the input statements of our object, to decide the optimizzation: \n    def __init__(self, classifier, params_grid = None, k_opt = 5, calibration = 0.2, compute_proba = \'model\'): \n        \n        # assign \n        self.classifier = classifier\n        self.params_grid = params_grid\n        self.params_win = None\n        #check if kopt is a positive integer\n        self.k_opt  = k_opt\n        \n        self.model = None\n        self.conf_matrix = None # the object "cm" request by the assignament. \n        self.calibrated_model = None \n        self.calibration = calibration\n        self.compute_proba = compute_proba\n        \n    #optimize: the best parameters in params grid for classifier are chosen via GridSearch according to \n    #k opt using (Xt\', Yt\'), such best parameters are stored in params win dictionary.\n    def __optimize(self,xt,yt):\n        #find opt params win\n        clf = None\n        if self.k_opt == 